<a href="https://colab.research.google.com/github/LuckyBoy587/Notes-Summarizer/blob/master/notebooks/notes_summarizer_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notes Summarizer (Colab)

This notebook is intentionally minimal: it only installs dependencies, clones the repository, and calls the project's `main.py`. No helper functions or UI code are embedded here.

Run the cells in order. After cloning, `main.py` will be invoked (it expects a PDF path argument; run it as shown or upload a file and adapt the command below).

In [ ]:
# Clone the repository and run main.py (this will print usage if no PDF path is provided)
!git clone https://github.com/LuckyBoy587/Notes-Summarizer.git || true
%cd Notes-Summarizer

!pip install -r requirements.txt
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')  

# Call main.py (it expects a path to a PDF). Without args it will print usage information.
!python main.py